In [21]:
import os.path
import numpy as np
import pandas as pd
from time import time

In [22]:
# set configure
path = "./"

path = "./marketing_analyze/"
# path = "D:/marketing_analyze_0715_backup/"
# path = "C:/Users/VIPLAB/Downloads/Spark_data/"
# filename = "april_marketing_anylze_full_table_group12_max_min"
# filename = "april_marketing_anylze_numeric"
# filename = "CHNL_NSP_CATG_MT_COMPLETED"

filename = "mkt_with_11_group_id_20170712"

# path = "D:/NEW_DATA_FULL_2017_6_16/"
# filename = "MDS_April_All_SINGLE_SIM_COMPLETED"
relative_filename = path + filename + ".csv"
# relative_filename = path + filename + ".txt"
# wants_cols = ['MINING_DW_SUBSCR_NO', 'CUST_TYPE']

In [23]:
# read csv file and print cost time
t0 = time()
# df = pd.read_csv(relative_filename, error_bad_lines=False)
# df = pd.read_csv(relative_filename, usecols = wants_cols, error_bad_lines=False)
# df = pd.read_csv(relative_filename, usecols = wants_cols, error_bad_lines=False, nrows = 100000)
# df = pd.read_csv(relative_filename, error_bad_lines=False)
# df = pd.read_csv(relative_filename, error_bad_lines=False)

# df = pd.read_csv(relative_filename, error_bad_lines=False)
# df = pd.read_csv(relative_filename, usecols = wants_cols, sep  = '\t', error_bad_lines=False)

df = pd.read_csv(relative_filename, error_bad_lines=False)
# df = pd.read_csv(relative_filename, error_bad_lines=False)
print("time for read csv: %.2f" % (time()-t0))


C:\ProgramData\Anaconda3\envs\obj_detect\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (24,28,76,97,98,99,100,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


time for read csv: 70.05


In [24]:
df_cols = list(df.columns)

In [25]:
len(df_cols)

306

In [26]:
df.head()

,DATA_MONTH,MINING_DW_SUBSCR_NO,CURR_SUBSCR_ID,DATA_MONTH_RPT_TEMP1,DATA_MONTH_RPT_TEMP2,DATA_MONTH_RPT,DW_PARTY_ID,RENEW_IND,GA_IND,ACTV_TYPE,...,L2M_DEVICE_PRICE_TIER,L2M_DEVICE_TYPE,HS_CHG_CNT,AVG_HS_USE_MONTH,AVG_HS_USE_MONTH_EX_CURR,ZONE_ACTV_IVR_COUNTRY,ZONE_ACTV_IVR_REGION,STORE_TYPE,SPAUTH_IND,Groups
0,2017/4/1,8134330931,327624391,2017/4/1,2017/4/1,2017/4/1,7508599,N,N,PURE GA,...,?,?,1,7.0,?,?,?,Mini-New(Y2013),N,InstantMessage-Low
1,2017/4/1,2118544651,340264445,2017/4/1,2017/4/1,2017/4/1,5638034,N,N,PURE GA,...,?,2G H/S,3,33.0,40,?,?,?,N,Infrequent_User
2,2017/4/1,10000337216,341414412,2017/4/1,2017/4/1,2017/4/1,6556494,N,N,MNP,...,?,?,?,?,?,高雄市,左營區,Store,N,No_ignificant_preference
3,2017/4/1,3133428916,326979481,2017/4/1,2017/4/1,2017/4/1,14170468,N,N,MNP,...,?,?,1,6.0,?,新北市,瑞芳區,Store,N,No_ignificant_preference
4,2017/4/1,3135493626,328506173,2017/4/1,2017/4/1,2017/4/1,1245630,N,N,MNP,...,?,?,?,?,?,?,?,Mini-New(Y2012),N,InstantMessage-Low


# Drop Column List

In [27]:
# saving money id bcz it will merge group
# MINING_DW_SUBSCR_NO

drop_str = """
CURR_SUBSCR_ID
DW_PARTY_ID
DATA_MONTH_RPT_TEMP1
DATA_MONTH_RPT_TEMP2
DATA_MONTH_RPT
GA_DATE
CHURN_IND
CHURN_TYPE
INACTV_DATE
MKT_CHURN_DATE
SUSPEND_IND
TARGET_OPR_ID
TEMP_INACTV_DATE
RPS_NAME
SUBSCR_STATUS_CODE
BILL_DISTRICT_NAME
CHURN_INDEX
IMEI_SMART_OS_FLAG
ACTV_CHANNEL_NAME
SHIPMENT_CHANNEL_ID
SHIPMENT_CHANNEL_CODE
SHIPMENT_CHANNEL_NAME
CCI_CHG_DATE
SEGMENT_NAME
NP_IN_DATE
BONDING_FLAG
CHURN_WELCOME_STAGE
PTY_NP_IN_IND
PTY_NP_OUT_IND
RENEW_APPLY_DATE
MDS_ELIGIBLE_FLAG
PROM_BUNDLE_VRP
PROM_BUNDLE_DRP
PROM_CURR_PROM_NAME
PROM_CURR_SYS_MODEL_IND
PROM_DEVICE_TYPE
PROM_DEVICE_BRAND
PROM_DEVICE_OS
PROM_DEVICE_MODEL
PROM_ACTV_PROM_CODE
PROM_ACTV_PROM_NAME
PROM_ACTV_PROM_CATG
PROM_CURR_APPLY_DATE
PROM_CURR_START_DATE
PROM_CURR_END_DATE
PROM_CURR_CONTR_EXP_DATE
L1M_PROM_CODE
L1M_PROM_NAME
L1M_PROM_COMMIT_MONTHS
CURR_PROM_TTL_CMF
CURR_PROM_CONTRACT_PERIOD
L1M_PROM_TTL_CMF
L1M_PROM_CONTRACT_PERIOD
DUM_PROM_CODE
GPRS_CAP_IND
BEST_VRP_DESC
BEST_DRP_DESC
SERVICE_APPLY_MOVIE_FLAG
SERVICE_APPLY_OMUSIC_FLAG
SERVICE_APPLY_READING_FLAG
HS_LEASE_FEATURE_START_DATE
HS_LEASE_FEATURE_END_DATE
HS_LEASE_CONTR_NAME
HS_LEASE_CONTR_START_DATE
HS_LEASE_CONTR_END_DATE
SMS30_FLAG
CURR_DATA_BILL_PLAN_START_DATE
CURR_DATA_BILL_PLAN_END_DATE
PRE_DATA_BILL_PLAN_ID
PRE_DATA_BILL_PLAN_NAME
PRE_DATA_BILL_PLAN_START_DATE
PRE_DATA_BILL_PLAN_END_DATE
DATA_BOOSTER_CURR_START_DATE
LIM_BILL_PLAN_NAME
L1M_DATA_BILL_PLAN_NAME
CURR_BILL_PLAN_START_DATE
EVER_PSTN_FEATURE_START_DATE
EVER_PSTN_FEATURE_END_DATE
EVER_PSTN_MONTH_APPLY_CANCEL_FLAG
DISC_TYPE_AM_IND
DISC_TYPE_AW_IND
DISC_TYPE_AD_IND
DISC_TYPE_DS_IND
DISC_TYPE_DM_IND
DISC_TYPE_DB_IND
DISC_TYPE_DO_IND
DISC_TYPE_DR_IND
DISC_TYPE_DU_IND
DISC_TYPE_DC_IND
DISC_TYPE_DP_IND
VIP_EXPIRY_DATE
VIP_BEFORE_EXP_MONTH
VIP_MSISDN_CHG_IND
PAY_PENALTY_DATE
DIRECT_STORE_PYMT_IND
CVS_PYMT_IND
VIRTUAL_CHANNEL_PYMT_IND
L3M_DIRECT_STORE_PYMT_IND
L3M_CVS_PYMT_IND
L3M_VIRTUAL_CHANNEL_PYMT_IND
HG_SEGMENT
P6M_MO_ONNET_CNT
P6M_MO_ONNET_DUR
P6M_MO_OFFNET_CNT
P6M_MO_OFFNET_DUR
P6M_MO_PSTN_CNT
P6M_MO_PSTN_DUR
L1M_CHANNEL_CHURN_INDEX
RETAIL_STORE_CHURN_FLAG
RETAIL_STORE_CHURN_SRV_DATE
RETAIL_STORE_CHURN_DEPUTY_FALG
RETAIL_STORE_CHURN_STOP_REASON
HS_USE_MONTH
CURR_DEVICE_MAKER
CURR_DEVICE_MODEL
CURR_DEVICE_PRICE_TIER
L1M_DEVICE_TENURE
L1M_DEVICE_MAKER
L1M_DEVICE_MODEL
L1M_DEVICE_PRICE_TIER
L1M_DEVICE_TYPE
L2M_DEVICE_TENURE
L2M_DEVICE_MAKER
L2M_DEVICE_MODEL
L2M_DEVICE_PRICE_TIER
L2M_DEVICE_TYPE
ZONE_ACTV_IVR_COUNTRY
ZONE_ACTV_IVR_REGION
STORE_TYPE
SPAUTH_IND
PTY_EVER_PO_CNT
PTY_EVER_PP_CNT
ANOTHER_ACTIVE_VD
ANOTHER_ACTIVE_D
PTY_CBU_EBU_PO_CNT
PTY_CBU_PO_CNT
PTY_CBU_PO_V_CNT
PTY_CBU_PO_VD_CNT
PTY_CBU_PO_D_CNT
PTY_EBU_PO_CNT
PTY_PO_ALL_CNT
L6M_AVG_NET_INV_AMT
L1M_NET_INV_AMT
L2M_NET_INV_AMT
W2P_SMS_MO_INT_CNT
SMS2P_SMS_MO_CNT
DATA_RATING_USAGE_MB
P6M_AVG_NET_INV_AMT
L1M_GPRS_AMT
L2M_GPRS_AMT
L6M_AVG_GPRS_AMT
L6M_AVG_VAS_MB
W2P_SMS_MO_CNT
W2P_SMS_MO_ONNET_CNT
W2P_SMS_MO_OFFNET_CNT
L1M_DATA_USAGE_MB
L2M_DATA_USAGE_MB
P6M_AVG_DATA_USAGE_MB
P1M_MO_ONNET_CNT
P1M_MO_ONNET_DUR
P1M_MO_OFFNET_CNT
P1M_MO_OFFNET_DUR
P1M_MO_PSTN_CNT
P1M_MO_PSTN_DUR
"""

In [28]:
drop_list = drop_str.split("\n")
drop_list = [x for x in drop_list if x != ""]
drop_list

['CURR_SUBSCR_ID',
 'DW_PARTY_ID',
 'DATA_MONTH_RPT_TEMP1',
 'DATA_MONTH_RPT_TEMP2',
 'DATA_MONTH_RPT',
 'GA_DATE',
 'CHURN_IND',
 'CHURN_TYPE',
 'INACTV_DATE',
 'MKT_CHURN_DATE',
 'SUSPEND_IND',
 'TARGET_OPR_ID',
 'TEMP_INACTV_DATE',
 'RPS_NAME',
 'SUBSCR_STATUS_CODE',
 'BILL_DISTRICT_NAME',
 'CHURN_INDEX',
 'IMEI_SMART_OS_FLAG',
 'ACTV_CHANNEL_NAME',
 'SHIPMENT_CHANNEL_ID',
 'SHIPMENT_CHANNEL_CODE',
 'SHIPMENT_CHANNEL_NAME',
 'CCI_CHG_DATE',
 'SEGMENT_NAME',
 'NP_IN_DATE',
 'BONDING_FLAG',
 'CHURN_WELCOME_STAGE',
 'PTY_NP_IN_IND',
 'PTY_NP_OUT_IND',
 'RENEW_APPLY_DATE',
 'MDS_ELIGIBLE_FLAG',
 'PROM_BUNDLE_VRP',
 'PROM_BUNDLE_DRP',
 'PROM_CURR_PROM_NAME',
 'PROM_CURR_SYS_MODEL_IND',
 'PROM_DEVICE_TYPE',
 'PROM_DEVICE_BRAND',
 'PROM_DEVICE_OS',
 'PROM_DEVICE_MODEL',
 'PROM_ACTV_PROM_CODE',
 'PROM_ACTV_PROM_NAME',
 'PROM_ACTV_PROM_CATG',
 'PROM_CURR_APPLY_DATE',
 'PROM_CURR_START_DATE',
 'PROM_CURR_END_DATE',
 'PROM_CURR_CONTR_EXP_DATE',
 'L1M_PROM_CODE',
 'L1M_PROM_NAME',
 'L1M_PROM_CO

In [29]:
for col in drop_list:
    try:
        df = df.drop(col, axis = 1)
    except:
        print("col = %s is not in this table" % col)

In [30]:
len(df.columns)

142

# replace space/? to random value

In [31]:
random_fill_str = """
GENDER_CODE
IMEI_MFG_NAME
IMEI_MKT_NAME
ZONE_ACTIVATION_IVR
ACTV_STORE_ID
MOST_MO_OPERATOR
MOST_MT_OPERATOR
PROM_CURR_EXP_MONTH_CNT
PROM_CURR_PROM_CODE
"""
random_fill_list = random_fill_str.split("\n")
random_fill_list = [x for x in random_fill_list if x != ""]
random_fill_list

['GENDER_CODE',
 'IMEI_MFG_NAME',
 'IMEI_MKT_NAME',
 'ZONE_ACTIVATION_IVR',
 'ACTV_STORE_ID',
 'MOST_MO_OPERATOR',
 'MOST_MT_OPERATOR',
 'PROM_CURR_EXP_MONTH_CNT',
 'PROM_CURR_PROM_CODE']

In [32]:
question_item = ["?", " "]
for col in random_fill_list:
    for q_item in question_item:
        q_item_count = len(df[df[col] == q_item])
        if q_item_count > 0:
            sample_list = df[df[col] != q_item].sample(q_item_count)
            df.loc[df[df[col] == q_item].index, col] = sample_list    
    print("? count =", len(df[df[col] == "?"]))
    print("space count =", len(df[df[col] == " "]))
    print(col, "done")

? count = 0
space count = 0
GENDER_CODE done
? count = 0
space count = 0
IMEI_MFG_NAME done
? count = 0
space count = 0
IMEI_MKT_NAME done
? count = 0
space count = 0
ZONE_ACTIVATION_IVR done
? count = 0
space count = 0
ACTV_STORE_ID done
? count = 0
space count = 0
MOST_MO_OPERATOR done
? count = 0
space count = 0
MOST_MT_OPERATOR done
? count = 0
space count = 0
PROM_CURR_EXP_MONTH_CNT done
? count = 0
space count = 0
PROM_CURR_PROM_CODE done


# fill up the specific value

In [35]:
# specific_fill_str = """
# PROM_CURR_PROMOTION_TYPE
# PROM_CURR_PROMOTION_SUB_TYPE
# PROM_ACTV_PROM_SUB_TYPE
# LAST_CHANNEL_TYPE
# IMEI_BAND
# IMEI_TYPE
# """
# specific_fill_str = specific_fill_str.split("\n")
# specific_fill_list = [x for x in specific_fill_list if x != ""]
# specific_fill_list

specific_dict = {
    "PROM_CURR_PROMOTION_TYPE" : "others",
    "PROM_CURR_PROMOTION_SUB_TYPE" : "others",
    "PROM_ACTV_PROM_SUB_TYPE" : "others",
    "LAST_CHANNEL_TYPE" : "others",
    "IMEI_TYPE" : "Smart Phone",
    "VIP_TENURE" : 0,
    "P3M_MO_PSTN_DUR" : 0,
    "HS_CHG_CNT" : 0,
    "AVG_HS_USE_MONTH_EX_CURR" : 0,
    "PAY_PENALTY_AMT" : 0,
#     "跟價格有關的負數補0" : 0
    
}

In [36]:
for col, value in specific_dict.items():
    df.loc[df[df[col] == "?"].index, col] = value   

# IMEI_BAND fill up 3G or 4G

In [38]:
col = "IMEI_BAND"
q_item_count = len(df[df[col] == "?"])
sample_list = df[(df[col] == "3G") | (df[col] == "4G")].sample(q_item_count)
df.loc[df[df[col] == "?"].index, col] = sample_list 

In [39]:
df.groupby("IMEI_BAND").count()

,DATA_MONTH,MINING_DW_SUBSCR_NO,RENEW_IND,GA_IND,ACTV_TYPE,SRV_TYPE_DESC,GENDER_CODE,ORIG_OPR_ID,AGE,TENURE_SCV,...,P3M_MO_ONNET_CNT,P3M_MO_ONNET_DUR,P3M_MO_OFFNET_CNT,P3M_MO_OFFNET_DUR,P3M_MO_PSTN_CNT,P3M_MO_PSTN_DUR,HS_CHG_CNT,AVG_HS_USE_MONTH,AVG_HS_USE_MONTH_EX_CURR,Groups
IMEI_BAND,,,,,,,,,,,,,,,,,,,,,
2G,1447,1447,1447,1447,1447,1447,1439,1447,1447,1447,...,1447,1447,1447,1447,1447,1447,1447,1447,1447,1447
3G,133855,133855,133855,133855,133855,133855,133592,133855,133855,133855,...,133855,133855,133855,133855,133855,133855,133855,133855,133855,133855
4G,1723516,1723516,1723516,1723516,1723516,1723516,1718815,1723516,1723516,1723516,...,1723516,1723516,1723516,1723516,1723516,1723516,1723516,1723516,1723516,1723516


# TODO:ORIG_OPR_ID merge telegram 

In [40]:
df.groupby("ORIG_OPR_ID").count()

,DATA_MONTH,MINING_DW_SUBSCR_NO,RENEW_IND,GA_IND,ACTV_TYPE,SRV_TYPE_DESC,GENDER_CODE,AGE,TENURE_SCV,BILL_CITY_NAME,...,P3M_MO_ONNET_CNT,P3M_MO_ONNET_DUR,P3M_MO_OFFNET_CNT,P3M_MO_OFFNET_DUR,P3M_MO_PSTN_CNT,P3M_MO_PSTN_DUR,HS_CHG_CNT,AVG_HS_USE_MONTH,AVG_HS_USE_MONTH_EX_CURR,Groups
ORIG_OPR_ID,,,,,,,,,,,,,,,,,,,,,
?,944908,944908,944908,944908,944908,944908,940285,944908,944908,944908,...,944908,944908,944908,944908,944908,944908,944908,944908,944908,944908
AP4,19884,19884,19884,19884,19884,19884,19879,19884,19884,19884,...,19884,19884,19884,19884,19884,19884,19884,19884,19884,19884
APW,60044,60044,60044,60044,60044,60044,60038,60044,60044,60044,...,60044,60044,60044,60044,60044,60044,60044,60044,60044,60044
ARC,3480,3480,3480,3480,3480,3480,3480,3480,3480,3480,...,3480,3480,3480,3480,3480,3480,3480,3480,3480,3480
CH3,227773,227773,227773,227773,227773,227773,227526,227773,227773,227773,...,227773,227773,227773,227773,227773,227773,227773,227773,227773,227773
CH4,72533,72533,72533,72533,72533,72533,72458,72533,72533,72533,...,72533,72533,72533,72533,72533,72533,72533,72533,72533,72533
CHM,64931,64931,64931,64931,64931,64931,64898,64931,64931,64931,...,64931,64931,64931,64931,64931,64931,64931,64931,64931,64931
MBT,2403,2403,2403,2403,2403,2403,2403,2403,2403,2403,...,2403,2403,2403,2403,2403,2403,2403,2403,2403,2403
QW3,642,642,642,642,642,642,639,642,642,642,...,642,642,642,642,642,642,642,642,642,642


# 補眾數欄位

In [47]:
col = "BILL_CITY_NAME"
df.loc[df[df[col] == "?"].index, col] = df[col].describe().top


In [49]:
col = "AGENCY_FLAG"
df.loc[df[df[col] == "?"].index, col] = df[col].describe().top


# 補平均值

In [56]:
col = "AGE"
avg = sum(df[df[col] != "?"][col].apply(np.int64)) / len(df[df[col] != "?"])
df.loc[df[df[col] == "?"].index, col] = avg


In [60]:
len(df.columns)

142

In [ ]:
df

In [ ]:
path = "./marketing_analyze/"
filename = "marketing_filter.csv"
# TAG K MEAN RESULT
GROUP_ID_FILE = path + filename
group_df = pd.read_csv(GROUP_ID_FILE, error_bad_lines=False, usecols = ["MINING_DW_SUBSCR_NO", "Groups"])
df_combine_group = pd.merge(group_df, df, on='MINING_DW_SUBSCR_NO', how='inner')